# 안녕하세요^^ 
# AI 프로젝트 - 원내비 도착시간 예측 모델링 과정에 오신 여러분을 환영합니다.
* 본 과정에서는 주변에서 마주할 수 있는 연속성을 가지는 수치를 예측할 수 있는 회귀문제를 예제코드를 통해서 기계학습(머신러닝 및 딥러닝)을 활용해서 해결할 것입니다.
* 네비게이션 주행데이터를 가지고 도착시간 정확도를 개선하는 과정을 통해 실전 AI 과제 수행에 있어 자신감을 가질 수 있는 기회가 되었으면 좋겠습니다.
* 또한 각 단계별로 드래곤볼을 찾는 미션을 통해 과정에 대한 이해를 도울 수 있도록 구성했습니다.
* AI문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

----

### [상황] 첫 번째 드래곤볼을 찾아 보겠습니다. 
* 실습문제를 해결할 때마다 단서가 주어지는데요. 
* 이 단서를 기억해두었다가 마지막에 드래곤볼을 찾을 때 활용하시면 되요.

In [ ]:
# 필요 라이브러리부터 설치할께요.
!pip install pandas requests seaborn gspread oauth2client

In [ ]:
# ID를 입력해주세요.
# ID는 aivle 이메일의 아이디 a로 시작하는 7글자(예를 들어 a011001)
ID = ""

---

## 기본전제
* 처음에 제공되는 기본/추가 데이터는 'raw_data' 폴더에 있습니다.

[기본 데이터]
* 학습데이터 : onenavi_train.csv(7월 20일에서 24일 사이의 수도권 3~15km 주행데이터)
* 평가데이터 : onenavi_evaluation.csv(7월 27일에서 31일 사이의 수도권 3~15km 주행데이터)

[추가 데이터]
* 주소(시군구)데이터 : onenavi_pnu.csv(주행데이터를 기준으로 출발지의 주소 정보, key : RID)
* 신호등(갯수)데이터 : onenavi_signal.csv(주행데이터를 기준으로 경로의 신호등 갯수, key : RID)
* 날씨데이터 : onenavi_weather.csv(주행데이터를 기준으로 해당 일자의 수도권 날씨 정보, key : RID)

### 변수 소개
* RID : RouteID(경로 아이디로 주행 시 생성)
* TIME_DEPARTUREDATE : 출발시간
* TIME_ARRIVEDATE : 도착시간
* A_DISTANCE : 실 주행 거리
* ET : 실제 주행 시간
* ETA : 원내비 예측 주행 시간
* ETAA : 예측 주행 시간 정확도

---

# 1. 데이터 불러오기
## 모든 AI프로젝트의 시작은 '데이터 불러오기' 부터라고 할 수 있습니다.
+ KeyPoint : 불러오고자 하는 데이터에 따라 자유롭게 변수로 지정할 수 있다.

### 가. 데이터 프레임을 불러오고 변수로 저장(여기서는 CSV 기준으로 진행)
* csv : pd.read_csv("파일이름. csv")
* txt : pd.read_csv("파일이름. csv", sep="구분자")
* xlsx : pd.read_excel('파일이름.xlsx')
* pickle : pd.read_pickle("파일이름.pkl") <br>
 [참고] pickle은 파이썬의 모든 객체를 파일로 저장할 수 있는 방법으로 DataFrame,List,Dict 등 모든 객체 저장 가능(특히 sklearn라이브러리를 통해 모델을 학습시키고, 저장할 때 많이 사용)

#### [실습문제1] 데이터 로딩
* Pandas 라이브러리를 활용해서 'onenavi_train.csv'파일을 'df' 변수에 저장하고 그 Shape을 확인하세요.
* 아래에 'Pandas'를 Import하고 'onenavi_train.csv'파일을 불러오고 Shape을 확인하는 소스코드를 작성하고 실행해보세요.
* 데이터 파일 로딩시 참고 사항 : 구분자(sep)는 '|'(파이프라인)
* 단서 : 데이터프레임의 열(변수) 갯수, Hint : Shape을 출력했을 때 나오는 2개의 숫자 중 2번째

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
import pandas as pd

df = pd.read_csv('raw_data/onenavi_train.csv', sep='|')

In [ ]:
# 데이터 프레임의 Shape을 확인합니다.
df.shape

In [ ]:
# 아래에 단서를 적어주세요.
단서1 = "7"# 단서1을 적어주세요.(문자형으로 적어주세요.)

### 나. 학습데이터와 평가데이터 합치기
#### 학습데이터(onenavi_train.csv)는 7월 20일에서 24일까지 자료입니다.
#### 평가데이터(onenavi_evaluation.csv)는 7월 27일에서 31일까지 자료입니다.

#### [실습문제2] 데이터 합치기
* Pandas 라이브러리를 활용해서 'onenavi_train.csv'파일과 'onenavi_evaluation.csv'를 합쳐서 'df_total' 변수에 저장하고 그 Shape을 확인하세요.
* 아래에 'onenavi_train.csv'와 'onenavi_evaluation.csv' 파일을 불러오고 pandas의 'concat' 함수를 활용해서 데이터를 합친 후, Shape을 확인하는 소스코드를 작성하고 실행해보세요.
* 데이터 파일 로딩시 참고 사항 : 구분자(sep)는 '|'(파이프라인)
* 단서 : 데이터프레임의 행(데이터) 갯수, Hint : Shape을 출력했을 때 나오는 2개의 숫자 중 1번째

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df2 = pd.read_csv('raw_data/onenavi_evaluation.csv', sep='|')
df_total = pd.concat([df, df2], axis=0)

In [ ]:
# 데이터 프레임의 Shape을 확인합니다.
df_total.shape

In [ ]:
# 아래에 단서를 적어주세요.
단서2 = "109642"# 단서2를 적어주세요.(문자형으로 적어주세요.)

---

# 2. 추가변수 생성
## 주어진 데이터만 가지고 모델링을 한다면 세상을 너무 만만하게 보고 있는 것입니다.
+ KeyPoint : 모델에 영향을 줄 수 있는 다양한 변수를 고민하고 기존 데이터와 Merge 할 수 있다.

#### 사실 추가변수 생성은 데이터 전처리 영역의 범위로 보는 경우도 있습니다.
#### 데이터 분석과 전처리는 '닭이 먼저냐? 달걀이 먼저냐?'와 같이 상호 인과관계를 가질 수 있습니다.
#### 다만 여기서는 교육의 원활한 진행을 위해 데이터 분석 이전에 진행하겠습니다.
#### 주행에 영향을 미치는 요소는 어떤 것이 있을까요? 저는 좌표정보와 경로의 신호등 여부를 생각해봤습니다.
#### 그렇다면 어떻게 변수화 할까요?
#### 좌표정보는 출발지의 좌표를 기준으로 시군구 단위를 주소정보로 신호등 여부는 경로 상의 신호등 갯수를 변수로 만들 수 있겠네요.
#### Route ID를 기준으로 미리 데이터 프레임을 만들어 두었습니다.

#### [실습문제3] 추가변수 생성하기
* Pandas 라이브러리를 활용해서 'onenavi_pnu.csv'파일과 'onenavi_signal.csv'를 불러오고 'df_total'와 합쳐보세요. 그리고 Shape을 확인하세요.
* 아래에 'onenavi_pnu.csv'와 'onenavi_signal.csv' 파일을 불러오고 pandas의 'merge' 함수를 활용해서 'df_total' 데이터와 합친 후, Shape을 확인하는 소스코드를 작성하고 실행해보세요.
* 데이터 파일 로딩시 참고 사항 : 구분자(sep)는 '|'(파이프라인)
* 단서 : 데이터프레임을 merge할 때 key값, Hint : merge 함수의 옵션 'on' 

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
pnu = pd.read_csv('raw_data/onenavi_pnu.csv', sep='|')
sig = pd.read_csv('raw_data/onenavi_signal.csv', sep='|')

df_total = pd.merge(df_total, pnu, on='RID')
df_total = pd.merge(df_total, sig, on='RID')

In [ ]:
# 데이터 프레임의 Shape을 확인합니다.

df_total.shape

In [ ]:
# 아래에 단서를 적어주세요.
단서3 = "RID"# 단서3을 적어주세요.(문자형으로 적어주세요.)

---

# 3. 데이터 분석하기
## 최적의 모델을 만들기위해 데이터를 입체적으로 바라보는 시도는 중요합니다.
+ KeyPoint : 데이터의 형태를 살펴보고 다양한 분석기법을 통해 모델링에 적합하도록 정제요소를 선별할 수 있다.
  * 관측치들의 패턴 탐색
  * 잘못된 자료들을 탐색
  * 변수들간의 관계 파악




## 우리는 시각화를 통해서 데이터를 분석할 예정입니다.
* 데이터 시각화(data visualization)는 데이터 분석 결과를 쉽게 이해할 수 있도록 시각적으로 표현하고 전달되는 과정을 말한다. 데이터 시각화의 목적은 도표(graph)라는 수단을 통해 정보를 명확하고 효과적으로 전달하는 것이다.(위키백과)
* 여기서는 몇 가지 실습을 통해 시각화를 통해 데이터 인사이트를 도출해볼 예정입니다.

In [ ]:
# 시각화 한글폰트 설정
import seaborn as sns

sns.set(font="NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

#### [실습문제4] 데이터 분포 알아보기_1
* 변수 df_total에서 'level1_pnu' 변수를 기준으로 CountChart를 그려보고 인사이트를 도출해보세요.
* 단서 : Seaborn에서 CountChart를 그리기 위한 함수명은? Hint : 'sns.ooooooooo'에서 'ooooooooo'

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
import matplotlib.pyplot as plt

sns.countplot(df_total['level1_pnu'])
plt.show()

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.(집중! 주어진 데이터 정의와 차이 나는 점을 찾아내보세요.)
# 1. 충청, 강원 등 수도권역에 포함되지 않는 지역이 데이터에 있음 -> 수도권 근방 지역이라면 가능하긴 함
# 2. 지역이 표기되지 않은 데이터가 존재
# 3. 

In [ ]:
# 아래에 단서를 적어주세요.
단서4 = "countplot"# 단서4를 적어주세요.(문자형으로 적어주세요.)






#### [실습문제5] 데이터 분포 알아보기_2
* 변수 df_total에서 'ET'(실제 걸린 시간) 변수를 기준으로 DistPlot(히스토그램)를 그려보고 인사이트를 도출해보세요.
* 단서 : 그려진 차트에서 Y축의 이름은? Hint : 그려진 차트를 잘 살펴보세요. Y축에 있는 'Doooooo'


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
sns.distplot(df_total['ET'])
plt.show()

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.(집중! 데이터 분포의 모양을 잘 살펴봐주세요.)
# 1. 대부분 약 30분 이전에 분포
# 2. 10~15분 내외에 높은 밀도로 데이터 존재
# 3. 13~14분 사이 갑자기 데이터 밀도가 줄어드는 경향

In [ ]:
# 아래에 단서를 적어주세요.
단서5 = "Density"# 단서5를 적어주세요.(문자형으로 적어주세요.)






#### [실습문제6] 데이터 분포 알아보기_3
* 변수 df_total에서 X축은 'level1_pnu', Y축은 'ET'(실제 걸린 시간) 변수를 기준으로 BoxPlot를 그려보고 인사이트를 도출해보세요.
* 단서 : 충청북도와 강원도, 충청남도의 이상치(Outlier)는 몇 개인가요? Hint : 無에서 왔다 無로 간다.


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

sns.boxplot(data=df_total, x='level1_pnu', y='ET')
plt.show()

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.
# 1. 경기도 및 서울 지역에서 출발한 데이터는 편차가 매우 큼(outlier가 매우 많음)
# 2. 서울 지역에서 출발하는 데이터는 타 지역보다 실제 이동시간 평균이 높음
# 3. 수도권역이 아닌 지역의 데이터는 매우 적음(4건)

In [ ]:
# 아래에 단서를 적어주세요.
단서6 = "0"# 단서6을 적어주세요.(문자형으로 적어주세요.)






#### [실습문제7] 데이터 분포 알아보기_4
* 현재의 변수 df_total를 기준으로 Pairplot를 그려보고 인사이트를 도출해보세요.
* 단서 : Pairplot으로 그릴 수 있는 변수는 몇 개인가요? Hint : Pairplot은 수치형 변수만 시각화 할 수 있어요.


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_total.info()

In [ ]:
sns.pairplot(df_total)

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.
# 1. 주행 시간이 짧을 수록 정확도가 높은 경향이 있음(주행 시간이 길 수록 정확도 낮은 경향 있음)
# 2. 주행 예측 시간이 길 수록 정확도가 높은 경향이 있음
# 3. 주행 거리가 길 수록 정확도가 낮은 비율이 줄어드는 경향이 있음

In [ ]:
# 아래에 단서를 적어주세요.
단서7 = "5"# 단서7을 적어주세요.(문자형으로 적어주세요.)

### 다. 상관관계 분석(Pandas/Seaborn)
##### 상관관계 분석은 왜 하는것일까요? 변수 간 관계를 들여다보기 위해서!
##### 막연하게 상관관계는 높으면 좋을 것 같습니다. 하지만 AI모델링 또는 통계분석에서 꼭 좋은 것은 아닙니다. 가령 살아온 날수와 식사를 한 끼니 수는 강한 상관관계를 가질 것입니다. 그러나 조금만 다르게 보면 둘은 같은 이야기를 하는 것이라고 할 수 있습니다.(독립변수 간에 상관관계가 너무 높은 경우 다중공선성의 문제 발생 : 해당 변수의 종속변수에 미치는 영향력을 올바로 설명하지 못하게 됨)
##### 물론 상관관계 계수만으로 어떤 결정을 한다는 것은 섣부른 일입니다. 어떤 변수인지 확인이 반드시 필요하며, 시각화를 통해 분포도 확인하고 결정을 해야합니다.
[다중공선성 해결 Tip]
* 상관관계가 높은 독립변수중 하나 혹은 일부를 제거
* 변수를 변형시키거나 새로운 관측치를 이용
* 기타 다른 분석을 이용








#### [실습문제8] 데이터 분포 알아보기_5
* 변수 df_total를 토대로 상관계수를 구하고 HeatMap으로 시각화 후 나름의 분석을 해보세요.
* 단서 : 'ETA'와 'ET'의 상관계수는 얼마인가요?(소수점 2째자리까지)


In [ ]:
# 아래에 실습코드를 작성하고 상관계수를 확인 및 시각화 합니다.
df_num = df_total.loc[:, ['A_DISTANCE', 'ET', 'ETA', 'ETAA', 'signaltype']]

df_num.corr()

In [ ]:
plt.figure(figsize = (8, 8))
sns.heatmap(df_num.corr(), annot = True, fmt = '.3f', cmap = 'RdYlBu_r',  vmin = -1, vmax = 1)
plt.show()

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.
# 1. 주행 거리와 주행 시간, 주행 예측 시간은 강한 양의 상관관계를 가진다.
# 2. 주행 시간과 주행 예측 시간은 강한 양의 상관관계를 가진다.
# 3. 예측 정확도는 주행 거리 또는 시간과 유의한 상관관계를 가지지 않는다.

In [ ]:
# 아래에 단서를 적어주세요.
단서8 = "0.84"# 단서8을 적어주세요.(문자형으로 적어주세요.)

---






### [드래곤볼을 찾아라] 주어진 단서를 모아서 드래곤볼을 찾아보겠습니다.
* 소스코드 실행 후 드래곤볼 이미지 창이 열리고 "드래곤볼을 획득했습니다."라는 문구가 출력된다면 여러분은 드래곤볼을 찾은 거랍니다.


In [ ]:
# 단서를 모두 모았다면 드래곤볼을 찾아봅시다! 아래 소스코드를 실행해 주세요.
import module

module.firstModule(ID, 단서1, 단서2, 단서3, 단서4, 단서5, 단서6, 단서7, 단서8)

In [ ]:
# 확인
from IPython.display import Image
Image("./dragonball/1성구.png", width=300, height=300)